In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import timm
from peft import get_peft_model, LoraConfig

# --- 1. CONFIG ---
class CFG:
    base_path = Path("/kaggle/input/csiro-biomass/")
    
# PFAD ANPASSEN!
    model_path = Path("/kaggle/input/image2biomassprediction-dinov3/best_dinov3_biomass_v2.pth")
    
    MODEL_NAME = "vit_huge_plus_patch16_dinov3.lvd1689m"
    IMG_SIZE = 512
    BATCH_SIZE = 8  # Für Inference geht 8 meistens, sonst 4
    
# Deine Faktoren (aus dem training davor)
    FACTOR_GREEN  = 1.0 
    FACTOR_DEAD   = 1.0
    FACTOR_CLOVER = 1.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 2. DIE LORA-KLASSE (Muss exakt wie im training sein) ---
class LoRABiomassModel(nn.Module):
    def __init__(self, backbone_name, pretrained=False): # Pretrained=False für Submission!
        super().__init__()
        
        self.backbone = timm.create_model(
            backbone_name,
            pretrained=pretrained, # WICHTIG: False
            num_classes=0,
            global_pool='', 
        )
        self.backbone.set_grad_checkpointing(False) 

        peft_config = LoraConfig(
            r=16, # Oder 8, falls du reduziert hattest
            lora_alpha=32, # Oder 16
            target_modules=["qkv", "proj", "fc1", "fc2"], # Oder nur ["qkv", "proj"] wenn du reduziert hast
            lora_dropout=0.05,
            bias="none",
            inference_mode=True # Hier True für Inference
        )
        
        self.backbone = get_peft_model(self.backbone, peft_config)
        
        if hasattr(self.backbone, "base_model"):
            nf = self.backbone.base_model.model.num_features
        else:
            nf = self.backbone.num_features

        self.pool = nn.AdaptiveAvgPool1d(1)
        
        self.fusion = nn.Sequential(
            nn.Linear(nf * 2, nf),
            nn.LayerNorm(nf),
            nn.GELU(),
            nn.Dropout(0.2)
        )
        
        self.head_green = nn.Linear(nf, 1)
        self.head_dead = nn.Linear(nf, 1)
        self.head_clover = nn.Linear(nf, 1)

    def forward(self, x):
        left, right = x
        x_l = self.backbone(left)
        x_r = self.backbone(right)
        
# Mean Pooling
        x_l = x_l.mean(dim=1) 
        x_r = x_r.mean(dim=1)
        
        x_cat = torch.cat([x_l, x_r], dim=1)
        feat = self.fusion(x_cat)
        
        green = nn.functional.softplus(self.head_green(feat))
        dead = nn.functional.softplus(self.head_dead(feat))
        clover = nn.functional.softplus(self.head_clover(feat))
        
        return torch.cat([green, dead, clover], dim=1)

# --- 3. DATALOADER ---
test_df_raw = pd.read_csv(CFG.base_path / "test.csv")
unique_test_images = test_df_raw['image_path'].unique()

class InferenceDataset(Dataset):
    def __init__(self, image_paths, image_root, transform=None):
        self.image_paths = image_paths
        self.image_root = image_root
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path_str = self.image_paths[idx]
        full_path = self.image_root / path_str
        img = Image.open(full_path).convert("RGB")
        w, h = img.size
        
        left_img  = img.crop((0, 0, h, h))
        right_img = img.crop((w - h, 0, w, h))

        if self.transform:
            left_img = self.transform(left_img)
            right_img = self.transform(right_img)

        return (left_img, right_img), path_str

# Keine Augmentation bei Test, nur Resize
val_tfms = T.Compose([
    T.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_ds = InferenceDataset(unique_test_images, CFG.base_path, transform=val_tfms)
test_loader = DataLoader(test_ds, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=0)

# --- 4. model LADEN ---
print(f"Lade LoRA-Modell von: {CFG.model_path}")

model = LoRABiomassModel(backbone_name=CFG.MODEL_NAME, pretrained=False)

# Da wir torch.save(model.state_dict()) gemacht haben, laden wir es einfach rein
state_dict = torch.load(CFG.model_path, map_location=device)
model.load_state_dict(state_dict)

model.to(device)
model.eval()

# --- 5. TTA INFERENCE LOOP (4x Rotation) ---
prediction_map = {}
print("Starte TTA Inference (4-fach Rotation)...")

with torch.no_grad():
    for (img_l, img_r), paths in tqdm(test_loader):
        img_l, img_r = img_l.to(device), img_r.to(device)
        
        tta_preds = []
        
# 4 Rotationen (0, 90, 180, 270 Grad)
        for k in range(4):
            rot_l = torch.rot90(img_l, k=k, dims=[2, 3])
            rot_r = torch.rot90(img_r, k=k, dims=[2, 3])
            
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                pred = model((rot_l, rot_r))
            tta_preds.append(pred)
        
# Durchschnitt bilden
        avg_preds = torch.stack(tta_preds).mean(dim=0)
        
        preds = avg_preds.cpu().numpy()
        preds[preds < 0] = 0
        
        for i, path in enumerate(paths):
            p_green = preds[i, 0] * CFG.FACTOR_GREEN
            p_dead  = preds[i, 1] * CFG.FACTOR_DEAD
            p_clover = preds[i, 2] * CFG.FACTOR_CLOVER
            
            prediction_map[path] = {
                "Dry_Green_g": p_green,
                "Dry_Dead_g": p_dead,
                "Dry_Clover_g": p_clover,
                "Dry_Total_g": p_green + p_clover + p_dead,
                "GDM_g": p_green + p_clover
            }

# --- 6. CSV ERSTELLEN ---
print("Erstelle submission.csv...")
submission_df = test_df_raw.copy()

def get_prediction(row):
    return prediction_map.get(row['image_path'], {}).get(row['target_name'], 0.0)

submission_df['target'] = submission_df.apply(get_prediction, axis=1)
final_sub = submission_df[['sample_id', 'target']]
final_sub.to_csv("submission.csv", index=False)


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Lade LoRA-Modell von: /kaggle/input/image2biomassprediction-dinov3/best_dinov3_biomass_v2.pth
Starte TTA Inference (4-fach Rotation)...


  0%|          | 0/1 [00:00<?, ?it/s]

Erstelle submission.csv...
✅ Fertig! Viel Erfolg!
